In [1]:
import os, sys, time, importlib
os.environ.setdefault("MUJOCO_GL", "egl")         # MuJoCo가 EGL 백엔드 사용
os.environ.setdefault("PYOPENGL_PLATFORM", "egl") # PyOpenGL도 EGL 강제

from datetime import datetime
from typing import Optional, Tuple

import numpy as np
import imageio
from tqdm import tqdm
import os
from glob import glob

import torch
from stable_baselines3 import SAC

# 프로젝트 루트 자동 등록 (이 노트북이 collectors/ 안에 있다고 가정)
NB_DIR = os.getcwd()
PROJ_ROOT = NB_DIR  # 노트북이 프로젝트 루트에 있으므로 그대로 사용
if PROJ_ROOT not in sys.path:
    sys.path.insert(0, PROJ_ROOT)


# 프로젝트 설정 모듈
from config import XML_DIR

# Gym wrappers
from gymnasium.wrappers import TimeLimit
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor


In [2]:
def make_env(env_module: str, xml_filename: Optional[str] = None, render_mode: Optional[str] = None, max_episode_steps: int = 2000):
    """
    프로젝트의 커스텀 환경: envs.<env_module>.ModularEnv(xml=..., render_mode=...)
    """
    mod = importlib.import_module(f"envs.{env_module}")
    EnvCls = getattr(mod, "ModularEnv")
    if xml_filename is None:
        xml_filename = os.path.join(XML_DIR, f"{env_module}.xml")
    env = EnvCls(xml=xml_filename, render_mode=render_mode)
    env = TimeLimit(env, max_episode_steps=max_episode_steps)
    return env


def wrap_vec_env(env, log_dir: Optional[str] = None):
    """
    SB3 호환을 위한 Monitor + DummyVecEnv 래핑(1-env).
    """
    if log_dir is not None:
        os.makedirs(log_dir, exist_ok=True)
        env = Monitor(env, filename=os.path.join(log_dir, "monitor.csv"))
    else:
        env = Monitor(env)
    return DummyVecEnv([lambda: env])


In [3]:
import numpy as np

def _to_cv2_ready(frame):
    """
    env.render()가 반환한 frame을 cv2.putText가 안전하게 처리할 수 있도록 정리:
    - dtype: uint8
    - 채널: 3채널(RGB) 보장 (RGBA면 알파 제거, 흑백이면 3채널로 반복)
    - 연속 메모리(C-contiguous)
    - writable 보장
    """
    arr = np.asarray(frame)

    # 채널/차원 정리
    if arr.ndim == 2:  # 흑백 -> 3채널
        arr = np.repeat(arr[..., None], 3, axis=2)
    elif arr.ndim == 3 and arr.shape[2] == 4:  # RGBA -> RGB
        arr = arr[..., :3]

    # dtype 정리
    if arr.dtype != np.uint8:
        if np.issubdtype(arr.dtype, np.floating):
            arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
        else:
            arr = arr.astype(np.uint8)

    # 연속/쓰기 가능 보장
    if not arr.flags.c_contiguous:
        arr = np.ascontiguousarray(arr)
    if not arr.flags.writeable:
        arr = arr.copy()

    return arr


In [4]:
def rollout_video_with_sac(
    env_name: str,
    checkpoint_path: str,
    xml: Optional[str] = None,
    steps: int = 2000,
    seed: int = 0,
    fps: int = 30,
    device: str = "auto",
    out_path: Optional[str] = None,
    overlay_return: bool = True,
    overlay_font_scale: float = 0.6,
    overlay_thickness: int = 1,
    break_on_episode_end: bool = True,
) -> str:
    """
    SB3 SAC 체크포인트(.zip/.pt)를 로드해 env를 실행하고 mp4(video)로 저장합니다.
    - env는 render_mode='rgb_array' 로 생성하여 프레임을 수집합니다.
    - overlay_return=True면 누적 리턴을 프레임 좌상단에 오버레이합니다.
    - break_on_episode_end=True면 첫 에피소드가 끝나면 영상 저장 후 종료합니다.

    Returns
    -------
    out_path : str
        저장된 동영상 경로 (mp4 또는 gif)
    """
    assert os.path.exists(checkpoint_path), f"Checkpoint not found: {checkpoint_path}"

    # 모델 로드
    model = SAC.load(checkpoint_path, device=device)

    # 캡처용 env
    env = make_env(env_name, xml_filename=xml, render_mode="rgb_array")
    # (선택) SB3 내부 state 기대를 위해 vec-env를 연결(필수는 아님, 실패해도 무시)
    try:
        model.set_env(wrap_vec_env(make_env(env_name, xml_filename=xml, render_mode=None)))
    except Exception:
        pass

    # 파일 경로
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    if out_path is None:
        data_dir = os.path.join(PROJ_ROOT, "data")
        os.makedirs(data_dir, exist_ok=True)
        out_path = os.path.join(data_dir, f"{env_name}_{ts}_inference.mp4")

    # 롤아웃
    frames = []
    cum_ret = 0.0
    obs, info = env.reset(seed=seed)

    # 오버레이를 위해 cv2가 있으면 사용, 없으면 텍스트 생략
    try:
        import cv2
        use_cv2 = True
    except Exception:
        use_cv2 = False
        overlay_return = False

    pbar = tqdm(total=steps, desc="Rollout", ncols=0)
    for t in range(steps):
        action, _ = model.predict(obs, deterministic=True)
        next_obs, reward, terminated, truncated, info = env.step(action)
        cum_ret += float(reward)

        frame = env.render()
        if frame is not None:
            try:
                frame = np.flipud(frame)  # 상하 반전된 경우 뒤집기
                frame = _to_cv2_ready(frame)  # <-- 새 헬퍼로 변환
                if overlay_return:
                    txt = f"step: {t}   return: {cum_ret:.3f}"
                    cv2.putText(frame, txt, (10, 24),
                                cv2.FONT_HERSHEY_SIMPLEX, overlay_font_scale,
                                (255, 255, 255), overlay_thickness, cv2.LINE_AA)
                    cv2.putText(frame, txt, (10, 24),
                                cv2.FONT_HERSHEY_SIMPLEX, overlay_font_scale,
                                (0, 0, 0), max(1, overlay_thickness - 1), cv2.LINE_AA)
            except Exception as e:
                print(f"[WARN] overlay 실패: {e}, 텍스트 없이 저장합니다.")
            frames.append(frame)

        done = bool(terminated or truncated)
        if done:
            if break_on_episode_end:
                pbar.update(1)
                break
            # 다음 에피소드로
            obs, info = env.reset()
            cum_ret = 0.0
        else:
            obs = next_obs

        pbar.update(1)
    pbar.close()

    # 저장
    if len(frames) == 0:
        env.close()
        raise RuntimeError("수집된 프레임이 없습니다. env.render('rgb_array') 지원 여부를 확인하세요.")

    # mp4 우선 시도, 실패 시 gif
    saved = False
    try:
        imageio.mimsave(out_path, frames, fps=fps)  # mp4는 imageio-ffmpeg 필요
        saved = True
    except Exception as e:
        print(f"[WARN] mp4 저장 실패: {e}")

    if not saved:
        alt = out_path.rsplit(".", 1)[0] + ".gif"
        imageio.mimsave(alt, frames, fps=fps)
        out_path = alt
        print(f"[OK] GIF로 저장: {out_path}")
    else:
        print(f"[OK] MP4 저장: {out_path}")

    env.close()
    return out_path


In [ ]:
# ===== 사용자 설정 =====
ENV_NAME = "walker_2_main"      # 예: "hopper_4", "walker_5_main"
XML_PATH = None                      # None이면 config.XML_DIR/<env>.xml 사용
CKPT_PATH = "./checkpoint/walker_2_main_best.pt"  # 또는 best_model.zip

STEPS   = 1000       # 촬영 스텝 수(첫 에피소드 끝나면 중단하도록 설정 가능)
SEED    = 88
FPS     = 30
DEVICE  = "cuda"     # "cuda", "cuda:0", "mps", "cpu", "auto"
OUT_MP4 = 'video/walker_2_main.mp4'       # None이면 data/ 폴더에 자동 생성
BREAK_ON_EP_END = True  # True면 첫 에피소드 종료 시 저장하고 끝냄


In [48]:
out = rollout_video_with_sac(
    env_name=ENV_NAME,
    checkpoint_path=CKPT_PATH,
    xml=XML_PATH,
    steps=STEPS,
    seed=SEED,
    fps=FPS,
    device=DEVICE,
    out_path=OUT_MP4,
    overlay_return=True,
    overlay_font_scale=0.6,
    overlay_thickness=2,
    break_on_episode_end=BREAK_ON_EP_END,
)
out


Rollout:   1% 14/1000 [00:00<00:07, 136.77it/s]

You are using a GLFW raw input patch. This is not the official GLFW library.


Rollout:  22% 221/1000 [00:01<00:04, 193.43it/s]


[OK] MP4 저장: video/walker_2_main.mp4


'video/walker_2_main.mp4'

# 모든 환경에 대해서 저장

In [5]:
# ===================== 사용자 설정 =====================
CHECKPOINT_DIR = "/dataset/usr012/jmpark/Moudlar_data/checkpoint"
STEPS = 1000
SEED = 88
FPS = 30
DEVICE = "cuda"
BREAK_ON_EP_END = True

# rollout_video_with_sac()은 이미 구현되어 있다고 가정
# from your_module import rollout_video_with_sac

# ======================================================

In [6]:
def main():
    os.makedirs("video", exist_ok=True)
    ckpt_files = sorted(glob(os.path.join(CHECKPOINT_DIR, "*_best.pt")))
    if not ckpt_files:
        print(f"[WARN] No *_best.pt found in {CHECKPOINT_DIR}")
        return

    for ckpt_path in ckpt_files:
        base = os.path.basename(ckpt_path)
        env_name = base.replace("_best.pt", "")
        out_path = os.path.join("video", f"{env_name}.mp4")

        print(f"\n[RUN] {env_name}")
        print(f" - ckpt: {ckpt_path}")
        print(f" - out:  {out_path}")

        try:
            out = rollout_video_with_sac(
                env_name=env_name,
                checkpoint_path=ckpt_path,
                xml=None,
                steps=STEPS,
                seed=SEED,
                fps=FPS,
                device=DEVICE,
                out_path=out_path,
                overlay_return=True,
                overlay_font_scale=0.6,
                overlay_thickness=2,
                break_on_episode_end=BREAK_ON_EP_END,
            )
        except Exception as e:
            print(f"[ERROR] Failed for {env_name}: {e}")
        else:
            print(f"[DONE] Saved video: {out_path}")

if __name__ == "__main__":
    main()


[RUN] cheetah_2_back
 - ckpt: /dataset/usr012/jmpark/Moudlar_data/checkpoint/cheetah_2_back_best.pt
 - out:  video/cheetah_2_back.mp4


Rollout: 100% 1000/1000 [00:03<00:00, 282.59it/s]


[OK] MP4 저장: video/cheetah_2_back.mp4
[DONE] Saved video: video/cheetah_2_back.mp4

[RUN] cheetah_2_front
 - ckpt: /dataset/usr012/jmpark/Moudlar_data/checkpoint/cheetah_2_front_best.pt
 - out:  video/cheetah_2_front.mp4


Rollout: 100% 1000/1000 [00:03<00:00, 297.67it/s]


[OK] MP4 저장: video/cheetah_2_front.mp4
[DONE] Saved video: video/cheetah_2_front.mp4

[RUN] cheetah_3_back
 - ckpt: /dataset/usr012/jmpark/Moudlar_data/checkpoint/cheetah_3_back_best.pt
 - out:  video/cheetah_3_back.mp4


...

[OK] MP4 저장: video/walker_6_main.mp4
[DONE] Saved video: video/walker_6_main.mp4

[RUN] walker_7_flipped
 - ckpt: /dataset/usr012/jmpark/Moudlar_data/checkpoint/walker_7_flipped_best.pt
 - out:  video/walker_7_flipped.mp4


Rollout: 100% 1000/1000 [00:04<00:00, 243.47it/s]


[OK] MP4 저장: video/walker_7_flipped.mp4
[DONE] Saved video: video/walker_7_flipped.mp4

[RUN] walker_7_main
 - ckpt: /dataset/usr012/jmpark/Moudlar_data/checkpoint/walker_7_main_best.pt
 - out:  video/walker_7_main.mp4


Rollout: 100% 1000/1000 [00:03<00:00, 262.16it/s]


[OK] MP4 저장: video/walker_7_main.mp4
[DONE] Saved video: video/walker_7_main.mp4
